In [ ]:
# 1. Önce paketleri kur
!pip install  peft accelerate bitsandbytes transformers datasets

# 2. Sonra model fine-tuning kodunu çalıştır
from datasets import load_dataset
from transformers import AutoTokenizer

from peft import LoraConfig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 33.9 MB/s eta 0:00:00


In [ ]:
!pip uninstall -U trl
!pip install -U trl==0.9.6

import trl
print(trl.__version__)


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -U
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-

0.9.6


In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
    AutoModelForCausalLM
)
from trl import SFTTrainer
from peft import LoraConfig
import torch

# Model ve Tokenizer
model_id = "TURKCELL/Turkcell-LLM-7b-v1"

# 4-bit quantization konfigürasyonu (A100 için BF16 ile)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,  # Çift quantization ile daha fazla bellek tasarrufu
    bnb_4bit_quant_type="nf4"  # Normal Float 4 quantization
)

# Quantization ile modeli önceden yükle
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
    low_cpu_mem_usage=True  # CPU bellek kullanımını optimize et
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # Padding token'ı ayarla
tokenizer.padding_side = 'right'  # SFTTrainer uyarısını gidermek için

# Dataset yükle
dataset = load_dataset("json", data_files="data.json", split="train")

# Formatting function
def formatting_func(example):
    instruction = example.get("instruction", "")
    input_text = example.get("input", "")
    output = example.get("output", "")

    if input_text:
        return f"Soru: {instruction}\nGirdi: {input_text}\nCevap: {output}"
    else:
        return f"Soru: {instruction}\nCevap: {output}"

dataset = dataset.map(lambda x: {"text": formatting_func(x)})

# LoRA config
lora_config = LoraConfig(
    r=16,  # Daha düşük rank değeri ile bellek kullanımını azalt
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # Daha fazla modül ekle
    bias="none",
    task_type="CAUSAL_LM"
)

# Optimize edilmiş Training args
training_args = TrainingArguments(
    output_dir="./outputs",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,  # Daha yüksek gradient accumulation
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    fp16=False,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",  # Daha iyi bellek yönetimi için
    report_to="none",
    max_grad_norm=0.3,
    warmup_ratio=0.03
)

# Trainer
trainer = SFTTrainer(
    model=model,  # Önceden yüklenmiş quantize edilmiş model
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=512,
    args=training_args
)

# CUDA bellek ayarları
torch.cuda.empty_cache()
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Eğitimi başlat
trainer.train()

# Modeli kaydet
trainer.save_model("./outputs")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2771 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/2771 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:413: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.028900
20,2.667800
30,2.426700
40,1.994900
50,2.005300
60,1.780700
70,1.851700
80,1.744000
90,1.561300
100,1.690600


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/

In [ ]:
from google.colab import drive

# Google Drive'ı /content/drive dizinine bağlar
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r /content/outputs /content/drive/MyDrive/outputsdposuz

In [ ]:
!pip install --upgrade transformers==4.46.2
!pip install --upgrade trl==0.11.4
!pip install --upgrade peft==0.11.1
!pip install --upgrade accelerate==0.33.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 138.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 110.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 25.9 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.9.6
    Uninstalling trl-0.9.6:
      Successfully uninstalled trl-0.9.6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.17.0
    Uninstalling peft-0.17.0:
      

In [ ]:
!pip install -U peft trl transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 135.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 108.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.

In [ ]:
# === DPO: SFT sonrası LoRA adapter'larıyla devam ===
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from trl import DPOConfig, DPOTrainer
import torch, os

# ---- Model yolları ----
model_id = "TURKCELL/Turkcell-LLM-7b-v1"
sft_adapters_dir = "/content/drive/MyDrive/outputsdposuz"      # SFT sonrası adapter'lar
dpo_output_dir   = "/content/drive/MyDrive/outputsdpolu"  # DPO çıktıları buraya

# ---- 4-bit quantization ----
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# ---- Tokenizer ----
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.padding_side = "right"
if tokenizer.pad_token is None and tokenizer.eos_token is not None:
    tokenizer.pad_token = tokenizer.eos_token

# ---- Politika modeli (trainable) ----
base_policy = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto"
)
policy_model = PeftModel.from_pretrained(base_policy, sft_adapters_dir, is_trainable=True)

# ---- Referans modeli (aynı SFT, donmuş) ----
base_ref = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto"
)
ref_model    = PeftModel.from_pretrained(base_ref, sft_adapters_dir, is_trainable=False)
for p in ref_model.parameters():
    p.requires_grad_(False)

# ---- DPO veri seti ----
# Beklenen kolonlar: "prompt", "chosen", "rejected"
dpo_dataset = load_dataset("json", data_files="dpo_dataset.json", split="train")
# Eğer farklı isimler varsa rename_columns kullan:
# dpo_dataset = dpo_dataset.rename_columns({"soru":"prompt","kazanan":"chosen","kaybeden":"rejected"})

# ---- DPO konfigürasyonu ----
dpo_args = DPOConfig(
    output_dir=dpo_output_dir,
    beta=0.1,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    fp16=False,
    max_length=1024,
    max_prompt_length=512,
    report_to=[]
)

# ---- CUDA ayarı ----
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

# ---- DPO Trainer (eski & yeni TRL uyumlu) ----
try:
    # Yeni TRL (>=0.11) → processing_class parametresi
    dpo_trainer = DPOTrainer(
        model=policy_model,
        ref_model=ref_model,
        train_dataset=dpo_dataset,
        args=dpo_args,
        processing_class=tokenizer
    )
except TypeError:
    # Eski TRL (<=0.10.x) → tokenizer parametresi
    dpo_trainer = DPOTrainer(
        model=policy_model,
        ref_model=ref_model,
        train_dataset=dpo_dataset,
        args=dpo_args,
        tokenizer=tokenizer
    )

# ---- Eğitim ve kayıt ----
dpo_trainer.train()
dpo_trainer.save_model(dpo_output_dir)     # LoRA adapter'ları kaydedilir
tokenizer.save_pretrained(dpo_output_dir)  # tokenizer da aynı klasöre
dpo_trainer.save_state()                   # (opsiyonel) trainer state

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting prompt in train dataset:   0%|          | 0/1454 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1454 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1454 [00:00<?, ? examples/s]

Step,Training Loss
10,0.267700
20,0.059700
30,0.013700
40,0.001900
50,0.003800
60,0.000500
70,0.002700
80,0.001700
90,0.000400
100,0.000600


In [ ]:
!cp -r /content/outputs_dpo /content/drive/MyDrive/outputs_dpo

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 82.8 MB/s eta 0:00:00


In [ ]:
# Bu hücrede RAG için FAISS indekslerini kuruyoruz
# İki ayrı indeks: factual ve procedural
# Embedding modeli: BGE-M3 (çok dilli)

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embed_model = SentenceTransformer("BAAI/bge-m3")

# Dataseti güvenli şekilde böl (metadata kontrolü ile)
factual_data = [
    ex for ex in dataset
    if ex and isinstance(ex, dict)
    and "metadata" in ex
    and isinstance(ex["metadata"], dict)
    and ex["metadata"].get("task_type") == "factual"
]

procedural_data = [
    ex for ex in dataset
    if ex and isinstance(ex, dict)
    and "metadata" in ex
    and isinstance(ex["metadata"], dict)
    and ex["metadata"].get("task_type") == "procedural"
]

print(f"Factual kayıt sayısı: {len(factual_data)}")
print(f"Procedural kayıt sayısı: {len(procedural_data)}")

# Embedding ve FAISS index builder
def build_index(data):
    texts = [ex["output"] for ex in data if "output" in ex]
    embeddings = embed_model.encode(texts, normalize_embeddings=True)
    index = faiss.IndexFlatIP(embeddings.shape[1])   # cosine similarity için normalize_embeddings=True + inner product
    index.add(np.array(embeddings))
    return index, texts

factual_index, factual_texts = build_index(factual_data)
procedural_index, procedural_texts = build_index(procedural_data)

print("RAG indexleri hazır! ✅")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

NameError: name 'dataset' is not defined

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from trl import DPOConfig, DPOTrainer
import torch, os

In [ ]:
model_id = "TURKCELL/Turkcell-LLM-7b-v1"

In [ ]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # Padding token'ı ayarla
tokenizer.padding_side = 'right'  # SFTTrainer uyarısını gidermek için

In [ ]:
# Bu hücrede LoRA'yı merge ediyoruz
from peft import AutoPeftModelForCausalLM
import torch

ft_model = AutoPeftModelForCausalLM.from_pretrained("/content/drive/MyDrive/outputsdpolu", device_map="auto", torch_dtype=torch.float16)
ft_model = ft_model.merge_and_unload()

ft_model.save_pretrained("/content/drive/MyDrive/mergedoutputsdpolu")
tokenizer.save_pretrained("/content/drive/MyDrive/mergedoutputsdpolu")
print("LoRA merge edildi, HuggingFace formatında kaydedildi.")


RuntimeError: Only a single TORCH_LIBRARY can be used to register the namespace prims; please put all of your definitions in a single TORCH_LIBRARY block.  If you were trying to specify implementations, consider using TORCH_LIBRARY_IMPL (which can be duplicated).  If you really intended to define operators for a single namespace in a distributed way, you can use TORCH_LIBRARY_FRAGMENT to explicitly indicate this.  Previous registration of TORCH_LIBRARY was registered at /dev/null:488; latest registration was registered at /dev/null:488

In [ ]:
!cp -r /content/merged_modeldpolu /content/drive/MyDrive/merged_modeldpolu

In [ ]:
!cp -r /content/outputs /content/drive/MyDrive/outputs